<a href="https://colab.research.google.com/github/Leerish/Deep-Learning-Notebooks/blob/main/Introduction_To_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to NLP Fundamentals in TensorFlow

NLP has the goal of deriving information out of natural language (could be sequences text or speech)

Another Common term for NLP problems is sequence to sequence problems or seq2seq

## Get Helper Functions



In [ ]:
#Importing Functions

!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

# Importing Functions needed for this section

from helper_functions import unzip_data , create_tensorboard_callback , plot_loss_curves , compare_historys

--2023-12-10 10:51:10--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py.1’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-12-10 10:51:10 (80.4 MB/s) - ‘helper_functions.py.1’ saved [10246/10246]



## Get a text dataset

Dataset we using is Kaggle's introduction to NLP dataset (text samples of Tweets labelled as disaster or not)

Original Source here:https://www.kaggle.com/code/philculliton/nlp-getting-started-tutorial



In [ ]:
! wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

#Unzip Data
unzip_data("nlp_getting_started.zip")

--2023-12-10 10:51:16--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.13.207, 74.125.26.207, 172.217.193.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.13.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.01s   

2023-12-10 10:51:16 (52.0 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



## Visualize the data

To visulaize the text sample , we first have to read them in , we can use pandas

In [ ]:
import pandas as pd

train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

train_data.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [ ]:
train_data["text"][0]

'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'

In [ ]:
# Shuffle training data

train_data_shuffled = train_data.sample(frac = 1 , random_state = 42)
train_data_shuffled

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0
...,...,...,...,...,...
5226,7470,obliteration,Merica!,@Eganator2000 There aren't many Obliteration s...,0
5390,7691,panic,NaN,just had a panic attack bc I don't have enough...,0
860,1242,blood,NaN,Omron HEM-712C Automatic Blood Pressure Monito...,0
7603,10862,NaN,NaN,Officials say a quarantine is in place at an A...,1


In [ ]:
# Test Data

test_data.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [ ]:
# How many examples are there
train_data_shuffled.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [ ]:
# Total Samples
len(train_data_shuffled) , len(test_data)

(7613, 3263)

In [ ]:
# visualize random samples

import random

random_index = random.randint(0 , len(train_data)-5)

for row in train_data_shuffled[["text" , "target"]][random_index : random_index+5].itertuples():
  _, text, target = row
  print(f"Target:{target}" ,"(real disaster)" if target > 0 else ("not a real disaster") )
  print(f"Text:\n{text}\n")
  print("---\n")

Target:0 not a real disaster
Text:
#SirensIcebreaker What is one fantasy work about a diverse heroine that you think everyone should read? https://t.co/HplJUr0OBo

---

Target:0 not a real disaster
Text:
Dead Space - Obliteration Imminent [2/2]: http://t.co/XJB0dCAaHf via @YouTube

---

Target:0 not a real disaster
Text:
She just said does he have a curfew 'nope'??

---

Target:1 (real disaster)
Text:
Dr. Bengston on #wildfire management: ÛÏnumbers and size of fires areas affected and costs of fighting them all show upward trend.Û #smem

---

Target:1 (real disaster)
Text:
#Glimpses: Hyderabad deluged by heavy rainfall | http://t.co/DctV1uJLHc http://t.co/QOx1jNQSAU

---



In [ ]:
### Split Data into training and validation set


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# Split training data into training and validation sets

train_sentences , val_sentences , train_labels , val_labels = train_test_split(train_data_shuffled["text"].to_numpy(),
                                                                               train_data_shuffled["target"].to_numpy(),
                                                                               test_size =0.1, # 10% for validation
                                                                               random_state = 42)


In [ ]:
# Check the lengths of each

len(train_sentences) , len(val_sentences) , len(train_labels) , len(val_labels)

(6851, 762, 6851, 762)

In [ ]:
train_sentences[:10] , train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

## Converting text into numbers

When dealing with text problems one of the first things you'll have to do beofre you can build a model is to convert your text into numbers

there are few ways
* Tokenization - Direct Mapping of numbers to words
* Embedding - Creating a matrix of feature vector

In [ ]:
import tensorflow as tf

from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

# Use default TextVectorization parameters

text_vectorizer = TextVectorization(max_tokens = 10000, # How many words in vocabulary (automatically <OOV>)
                                    standardize = "lower_and_strip_punctuation",
                                    split="whitespace",
                                    ngrams =None, # Create group of n-words
                                    output_mode = "int",
                                    output_sequence_length = None, # How Long is your sequence
                                    pad_to_max_tokens =True
                                    )



In [ ]:
len(train_sentences[0].split())

7

In [ ]:
# Find the average number of tokens (words) in training tweets

round(sum([len(i.split()) for i in train_sentences]) / len(train_sentences))

15

In [ ]:
# Set up text vectorization variables

max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length = 15 # max length our sequences will be ( e.g. how many words from a tweet a model sees)

text_vectorizer = TextVectorization(max_tokens = max_vocab_length,
                                    output_mode = "int",
                                    output_sequence_length = max_length)

In [ ]:
# Fit the text vectorizer to the training set

text_vectorizer.adapt(train_sentences)

In [ ]:
# Create a Sample Sentence and a tokenizer

sample_sentence = "Streets are flooded here"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[1585,   22, 2995,  140,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0]])>

In [ ]:
# Choose random sentences from dataset

random_sentence = random.choice(train_sentences)

print(f"Original Sentence:{random_sentence}\
          \n\nVectorized Version:")
text_vectorizer([random_sentence])

Original Sentence:Pandemonium In Aba As Woman Delivers Baby Without Face (Photos) @... http://t.co/JbxBi93CLu          

Vectorized Version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 502,    4, 1157,   26,  410, 1146,  469,  228,  289,  729,    1,
           0,    0,    0,    0]])>

In [ ]:
# Get the unique words in our vocabulary

words_in_vocab = text_vectorizer.get_vocabulary() # Get all of the words in the training data

top_5_words = words_in_vocab[:5] # Get the common words

bottom_5_words = words_in_vocab[-5:] # Least common words

print(f"Words in vocab:{len(words_in_vocab)}")
print(f"Most common words:{top_5_words}")
print(f"Least Common words:{bottom_5_words}")

Words in vocab:10000
Most common words:['', '[UNK]', 'the', 'a', 'in']
Least Common words:['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


### Creating an Embedding using an Embedding Layer

To make a Embedding we are using tensorflow's embedding layer

The parameters we care most are:
1. `input_dim` = size of vocabulary
2. `output_dim` = the size of output embedding vector
3. `input_length` = length of sequences beign passed to the embedding layer

In [ ]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim = max_vocab_length, # Set input shape
                             output_dim = 128,# output shape
                             input_length  = max_length # How long is each input
                             )
embedding

In [ ]:
# get a random sentence from training set

random_sentence = random.choice(train_sentences)
print(f"Original sentence:{random_sentence}\
          \n\nEmbedded Sentence:")
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original sentence:I remember when I worked at Mcdonalds I use to be hours late because we used military time and I use to mess up when I had to be there :/          

Embedded Sentence:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.0270978 , -0.016222  ,  0.04799818, ..., -0.00868781,
         -0.0289979 ,  0.02038974],
        [-0.04063693,  0.01749847, -0.02291824, ...,  0.00644368,
         -0.04998794,  0.02937203],
        [-0.0270672 ,  0.04351275, -0.02442666, ..., -0.04577438,
         -0.00203218,  0.04256829],
        ...,
        [-0.01775253, -0.04846703,  0.04523103, ..., -0.0364344 ,
          0.02872808, -0.02196536],
        [ 0.01807965,  0.0182367 , -0.04038263, ...,  0.03628352,
         -0.0320457 , -0.00241355],
        [ 0.01423221,  0.01822606,  0.02593639, ..., -0.01235114,
          0.02675981, -0.00095417]]], dtype=float32)>

## Modelling a text dataset and running a series of experiment

We have turned text sequences to numbers , we are start modelling

Experiments:
* Model 0: Baseline Model Naive Bayes Model
* Model 1: Feed-Forward Neural Network
* Model 2: LSTM model (RNN)
* Model 3: GRU Model (RNN)
* Model 4: Bidirectional-LSTM model
* Model 5: 1D Convulutional Model (CNN)
* Model 6: TensorFlow Hub Pretrained Feature Extractor
* Model 7: same as model 6 but with 10% data


Steps:
* Create a model
* Compile a model
* Fit a model
* Evaluate a model


### Model 0: Getting a Baseline

For the baseline model we will be using Sklearn's Multinomial Naive Bayes using TF-IDF formula to convert our words to numbers.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.naive_bayes import MultinomialNB

from sklearn.pipeline import Pipeline

# Create tokenization

model_0 = Pipeline([
    ("tfidf" , TfidfVectorizer()) , # Convert words to numbers using tfidf
    ("clf" , MultinomialNB()) # Model the text
])

# Fit the pipeline to training data

model_0.fit(train_sentences , train_labels)

In [ ]:
# Evaluate our model
baseline_score = model_0.score(val_sentences , val_labels)
print(f"Baseline Accuracy:{baseline_score*100:.2f}%")

In [ ]:
# Make Predictions

baseline_preds = model_0.predict(val_sentences)
baseline_preds[:10]

### Creating an evaluation function for our model experiments

we could evaluate our model's prediction with different metrics

Metrics:
* Accuracy
* Precision
* Recall
*F1-Score


In [ ]:
# Function to evaluate

from sklearn.metrics import accuracy_score , precision_recall_fscore_support

def cal_results(y_true , y_pred):
  """
  Calculates the model accuracy , precision , recall and F1 score of binary classification model
  """

  # Calculate accuracy
  model_accuracy = accuracy_score(y_true , y_pred)*100

  # Calculate model precision , recall and f1-score

  model_precision , model_recall , model_f1 , _ = precision_recall_fscore_support(y_true , y_pred ,average="weighted")

  model_results = { "accuracy" : model_accuracy,
                    "precision": model_precision,
                    "recall" : model_recall,
                    "f1" : model_f1}
  return model_results

In [ ]:
baseline_results = cal_results(y_true = val_labels , y_pred = baseline_preds)
baseline_results

NameError: ignored

### Model 1: Simple Dense Model

In [ ]:
# Create a tensorboard callback

from helper_functions import create_tensorboard_callback

# Create a directory to save tensorboard logs

SAVE_DIR = "model_logs"


In [ ]:
# Build model with Functional API

from tensorflow.keras import layers

inputs = layers.Input(shape=(1,) , dtype=tf.string) # 1-D string inputs
x =text_vectorizer(inputs) # Turn text to numbers
x = embedding(x) # Create an Embedding of numberized inputs
x = layers.GlobalAveragePooling1D()(x) # Condense the feature vector for each token
outputs = layers.Dense(1 ,activation ="sigmoid")(x)
model_1 = tf.keras.Model(inputs , outputs , name ="Model_1")

In [ ]:
model_1.summary()

In [ ]:
model_1.compile(loss="binary_crossentropy",
                optimizer = tf.keras.optimizers.Adam(),
                metrics=["accuracy"])


In [ ]:
history_1 = model_1.fit(x =train_sentences,
                        y = train_labels,
                        epochs = 5,
                        validation_data = (val_sentences , val_labels),
                        callbacks =[create_tensorboard_callback(dir_name = SAVE_DIR ,
                                                                experiment_name ="model_1_results")])

In [ ]:
# Check the results

model_1.evaluate(val_sentences , val_labels)

In [ ]:
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs.shape

In [ ]:
model_1_pred_probs[0]

In [ ]:
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))

In [ ]:
model_1_preds[:20]

In [ ]:
# Model_1 results

result_1 = cal_results(y_true = val_labels,
                       y_pred = model_1_preds)
result_1

### Visualizing embeddings



In [ ]:
# Get the vocabulary from text vectorization layer

words_in_vocab = text_vectorizer.get_vocabulary()

len(words_in_vocab) , words_in_vocab[:10]

In [ ]:
model_1.summary()

In [ ]:
# Get the weight matrix of embedding layer
# These are numerical representations of each token in our training data , that our model learned for 5 epochs

embed_wts = model_1.get_layer("embedding").get_weights()
embed_wts

In [ ]:
# Create embedding files

import io

out_v = io.open('vectors.tsv' , 'w' , encoding = 'utf-8')
out_m = io.open('metadata.tsv' ,'w', encoding='utf-8')

for index , word in enumerate(words_in_vocab):
  if index != 0:
    continue
  vec = embed_wts[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

## Recurrent Nueral Networks (RNN)

RNN's are useful for sequence data
the premise of recurrent nueral network is to use the representation of previous input to aid the representation of a later input

### Model 2: LSTM

LSTM : Long Short Term Memory ( One of the most popular LSTM Cells)

Our Structure of RNN is:

```
Input(Text) -> Tokenize -> Embedding -> Layers(RNN's/Dense) -> Output
```

In [ ]:
# Create an LSTM Model

from tensorflow.keras import layers

inputs = layers.Input(shape=(1,) , dtype="string")
x = text_vectorizer(inputs)
x= embedding(x)
#x = layers.LSTM(64 , return_sequences = True)(x) # When you are stacking RNN Cells together , we need to return sequences
x = layers.LSTM(64)(x)
#x = layers.Dense(64 , activation="relu")(x)
outputs = layers.Dense(1 , activation ="sigmoid")(x)

model_2 = tf.keras.Model(inputs , outputs , name="model_2_LSTM")



In [ ]:
model_2.summary()

In [ ]:
# compile the model

model_2.compile(loss ="binary_crossentropy",
                optimizer = tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [ ]:
history_2 = model_2.fit(train_sentences ,
                        train_labels ,
                        epochs = 5 ,
                        validation_data =(val_sentences , val_labels),
                        callbacks=[create_tensorboard_callback(SAVE_DIR , "model_2_LSTM")])

In [ ]:
# Make predictions

model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs[:10]

In [ ]:
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

In [ ]:
model_2_results = cal_results(y_true = val_labels,
                              y_pred = model_2_preds)
model_2_results

### Model 3: GRU

Another popular and effective RNN Component is the GRU or gated recurrent unit.

The GRU cell has similar features to an LSTM cell but has less parameters

In [ ]:
# build an RNN using GRU Cell

from tensorflow.keras import layers

inputs = layers.Input(shape=(1,) , dtype ="string")

x = text_vectorizer(inputs)
x= embedding(x)
x = layers.GRU(64 )(x)
#x= layers.LSTM(64 , return_sequences=True)(x)
#x=layers.GRU(64)(x)
#x = layers.Dense(64 , activation ="relu")(x)
outputs = layers.Dense(1 , activation ="sigmoid")(x)
model_3 = tf.keras.Model(inputs , outputs , name ="GRU")

In [ ]:
model_3.summary()

In [ ]:
# Compile the model

model_3.compile(loss = "binary_crossentropy",
                optimizer = tf.keras.optimizers.Adam(),
                metrics =["accuracy"])


In [ ]:
# fitting the model
history_3 = model_3.fit(train_sentences,
                        train_labels,
                        epochs =5,
                        validation_data=(val_sentences , val_labels),
                        callbacks=[create_tensorboard_callback(SAVE_DIR , "Model_GRU")])

In [ ]:
model_3_pred_probs =model_3.predict(val_sentences)
model_3_pred_probs[:10]

In [ ]:
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))
model_3_preds[:10]

In [ ]:
result_3 = cal_results(val_labels , model_3_preds)
result_3

### Model 4: Bi-Directional RNN

Normal RNN goes from left to right ( just like english ), however a bidirectional rnn goes from right to left as well as from left to right

In [ ]:
# Build a Bidirectional RNN

from tensorflow.keras import layers

inputs = layers.Input(shape=(1,) , dtype ="string")
x = text_vectorizer(inputs)
x = embedding(x)
#x = layers.Bidirectional(layers.LSTM(64 , return_sequences = True))(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
outputs =layers.Dense(1 , activation="sigmoid")(x)

model_4 = tf.keras.Model(inputs , outputs , name="Bidirectional_Model")



In [ ]:
model_4.summary()

In [ ]:
# Compile the model
model_4.compile(loss = "binary_crossentropy",
                optimizer = tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [ ]:
# Fitting the model

history_4 = model_4.fit(train_sentences , train_labels,
                        epochs = 5,
                        validation_data=(val_sentences , val_labels),
                        callbacks=[create_tensorboard_callback(SAVE_DIR ,"BiDirectional_expt")])

In [ ]:
# Make Predictions

model_4_pred_probs = model_4.predict(val_sentences)
model_4_pred_probs[:10]

In [ ]:
model_4_preds = tf.squeeze(tf.round(model_4_pred_probs))
model_4_preds[:10]

In [ ]:
result_4 = cal_results(val_labels , model_4_preds)
result_4

## Convolutional Nueral Networks for Text(and other types of sequences)

We've used CNNs for images that are typically 2D however text data is 1D

Structure of Conv1D model for sequences(text):
```
Inputs(Text) -> Tokenization -> Embedding -> (Conv Layer + Pooling Layer) -> Outputs
```

## Model 5 : Conv1D


In [ ]:
# Test out our Embedding layer , Conv1D layer and max pooling

embedding_test = embedding(text_vectorizer(["This is a test sentence"]))

conv_1d = layers.Conv1D(filters = 5, # Window moving over pixels
                        kernel_size = 5, # Window size
                        activation="relu",
                        padding="same") # Add addition layer around data to learn precisely (all tokens / pixels are learnt)
conv1d_output = conv_1d(embedding_test) # pass test embedding to 1D layer

max_pool = layers.GlobalMaxPool1D()

max_pool_op = max_pool(conv1d_output)

embedding_test.shape , conv1d_output.shape , max_pool_op.shape

(TensorShape([1, 15, 128]), TensorShape([1, 15, 5]), TensorShape([1, 5]))

In [ ]:
embedding_test

<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.00423698,  0.02227069,  0.02384636, ...,  0.0218623 ,
          0.03958369,  0.00935867],
        [ 0.02098681, -0.0334108 , -0.01563592, ...,  0.04684431,
          0.00049212, -0.04778397],
        [-0.04004949, -0.02208656, -0.03707272, ...,  0.02994767,
          0.00587621,  0.00197526],
        ...,
        [-0.01120061, -0.00845319, -0.04330715, ..., -0.01593503,
          0.04587313, -0.02642664],
        [-0.01120061, -0.00845319, -0.04330715, ..., -0.01593503,
          0.04587313, -0.02642664],
        [-0.01120061, -0.00845319, -0.04330715, ..., -0.01593503,
          0.04587313, -0.02642664]]], dtype=float32)>

In [ ]:
conv1d_output

<tf.Tensor: shape=(1, 15, 5), dtype=float32, numpy=
array([[[0.01783403, 0.0203228 , 0.02826201, 0.        , 0.01181908],
        [0.0323307 , 0.        , 0.        , 0.03943135, 0.02038452],
        [0.        , 0.02171658, 0.01519064, 0.02257488, 0.05425439],
        [0.00615651, 0.        , 0.0605317 , 0.        , 0.        ],
        [0.00622331, 0.        , 0.        , 0.        , 0.00524897],
        [0.        , 0.        , 0.02742286, 0.        , 0.04128202],
        [0.06073133, 0.        , 0.        , 0.        , 0.        ],
        [0.00680784, 0.        , 0.05633382, 0.        , 0.        ],
        [0.00680784, 0.        , 0.05633382, 0.        , 0.        ],
        [0.00680784, 0.        , 0.05633382, 0.        , 0.        ],
        [0.00680784, 0.        , 0.05633382, 0.        , 0.        ],
        [0.00680784, 0.        , 0.05633382, 0.        , 0.        ],
        [0.00680784, 0.        , 0.05633382, 0.        , 0.        ],
        [0.01268597, 0.        , 0.032

In [ ]:
max_pool_op

<tf.Tensor: shape=(1, 5), dtype=float32, numpy=
array([[0.06073133, 0.02171658, 0.0605317 , 0.03943135, 0.05425439]],
      dtype=float32)>

## Model 5: Conv1D

In [ ]:
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,) , dtype="string")
x = text_vectorizer(inputs)
x=embedding(x)
x = layers.Conv1D(filters = 64, # Window moving over pixels
                        kernel_size = 5, # Window size
                        activation="relu",
                        strides=1,
                        padding="valid")(x)
x = layers.GlobalMaxPool1D()(x)
outputs = layers.Dense(1 , activation="sigmoid")(x)

model_5 = tf.keras.Model(inputs , outputs , name ="ConV1D")

In [ ]:
model_5.summary()

Model: "ConV1D"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 conv1d_3 (Conv1D)           (None, 11, 64)            41024     
                                                                 
 global_max_pooling1d_3 (Gl  (None, 64)                0         
 obalMaxPooling1D)                                               
                                                                 
 dense_2 (Dense)             (None, 1)                 65   

In [ ]:
# Compile the model

model_5.compile(loss = "binary_crossentropy",
                optimizer = tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [ ]:
# Fit the model
history_5 = model_5.fit(train_sentences,train_labels,
                        epochs = 5 ,
                        validation_data=(val_sentences , val_labels),
                        callbacks=[create_tensorboard_callback(SAVE_DIR ,"CONV_EXPT")])

Saving TensorBoard log files to: model_logs/CONV_EXPT/20231210-110228
Epoch 1/5
215/215 [==============================] - 9s 34ms/step - loss: 0.5520 - accuracy: 0.7179 - val_loss: 0.4775 - val_accuracy: 0.7782
Epoch 2/5
215/215 [==============================] - 5s 22ms/step - loss: 0.3282 - accuracy: 0.8663 - val_loss: 0.4613 - val_accuracy: 0.7913
Epoch 3/5
215/215 [==============================] - 5s 25ms/step - loss: 0.1982 - accuracy: 0.9288 - val_loss: 0.5454 - val_accuracy: 0.7795
Epoch 4/5
215/215 [==============================] - 6s 26ms/step - loss: 0.1245 - accuracy: 0.9568 - val_loss: 0.6247 - val_accuracy: 0.7612
Epoch 5/5
215/215 [==============================] - 5s 22ms/step - loss: 0.0880 - accuracy: 0.9696 - val_loss: 0.6823 - val_accuracy: 0.7612


In [ ]:
model_5_pred_probs = model_5.predict(val_sentences)
model_5_pred_probs[:10]

24/24 [==============================] - 0s 6ms/step


array([[0.4738186 ],
       [0.25360745],
       [0.9993443 ],
       [0.069586  ],
       [0.00198872],
       [0.9981958 ],
       [0.8815244 ],
       [0.9960533 ],
       [0.99959666],
       [0.1749874 ]], dtype=float32)

In [ ]:
model_5_preds =tf.squeeze(tf.round(model_5_pred_probs))
model_5_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 0., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [ ]:
result_5 = cal_results(val_labels ,model_5_preds )
result_5

{'accuracy': 76.11548556430446,
 'precision': 0.7625228137873495,
 'recall': 0.7611548556430446,
 'f1': 0.7589888106092002}

## Model 6 : TensorFlow Hub Pretrained Sentence Encoder

We here use Universal Sentence Encoder : https://tfhub.dev/google/universal-sentence-encoder/4

In [ ]:
import tensorflow_hub as hub

embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
embed_samples = embed([sample_sentence ,
                       "Turns sentence into numbers"])
print(embed_samples[0][:50])

tf.Tensor(
[-0.07426048  0.02303515 -0.00527922  0.00731982  0.05479318  0.03481551
  0.01794909  0.06563365 -0.07663119  0.0243662  -0.00620972 -0.02544671
 -0.00828498  0.03377556  0.03416667 -0.03136539  0.00634929  0.01561056
 -0.03359335 -0.05389649  0.02562824  0.06504986 -0.03539924 -0.02787218
  0.00252122 -0.03228377  0.07160664  0.01392768 -0.0384841   0.04222204
  0.00169669  0.01753705 -0.03695782  0.03831246 -0.01062066 -0.05037636
  0.04957304  0.06192518 -0.01860108 -0.07575813  0.0643138  -0.01345967
 -0.05236956 -0.03017076 -0.09504528 -0.0549739  -0.01944733  0.05981122
 -0.03772434  0.00565695], shape=(50,), dtype=float32)


In [ ]:
embed_samples[0].shape

TensorShape([512])

In [ ]:
# Create a Keras layer using USE pretrained layer from tensorflow hub

encode_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                              input_shape = [],
                              dtype = tf.string,
                              trainable = False ,
                              name ="EncoderLayer")

In [ ]:
# Create model using Sequential API
model_6 = tf.keras.Sequential([
    encode_layer,
    layers.Dense(64 , activation="relu"),
    layers.Dense(1, activation="sigmoid")
],name ="Model_6_USE")

# Compile the model
model_6.compile(loss ="binary_crossentropy",
                optimizer = tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [ ]:
model_6.summary()

Model: "Model_6_USE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 EncoderLayer (KerasLayer)   (None, 512)               256797824 
                                                                 
 dense_5 (Dense)             (None, 1)                 513       
                                                                 
Total params: 256798337 (979.61 MB)
Trainable params: 513 (2.00 KB)
Non-trainable params: 256797824 (979.61 MB)
_________________________________________________________________


In [ ]:
# Train a model

history_6 = model_6.fit(train_sentences,
                        train_labels,
                        epochs=5,
                        validation_data=(val_sentences, val_labels),
                        callbacks=[create_tensorboard_callback(SAVE_DIR , "USE_EXPT")])

Saving TensorBoard log files to: model_logs/USE_EXPT/20231210-114601
Epoch 1/5
215/215 [==============================] - 8s 20ms/step - loss: 0.5029 - accuracy: 0.7900 - val_loss: 0.4477 - val_accuracy: 0.8018
Epoch 2/5
215/215 [==============================] - 3s 14ms/step - loss: 0.4152 - accuracy: 0.8148 - val_loss: 0.4424 - val_accuracy: 0.8097
Epoch 3/5
215/215 [==============================] - 3s 14ms/step - loss: 0.4015 - accuracy: 0.8221 - val_loss: 0.4332 - val_accuracy: 0.8163
Epoch 4/5
215/215 [==============================] - 4s 18ms/step - loss: 0.3927 - accuracy: 0.8247 - val_loss: 0.4294 - val_accuracy: 0.8150
Epoch 5/5
215/215 [==============================] - 4s 18ms/step - loss: 0.3860 - accuracy: 0.8288 - val_loss: 0.4295 - val_accuracy: 0.8202


In [ ]:
model_6_pred_probs = model_6.predict(val_sentences)
model_6_pred_probs[:10]

24/24 [==============================] - 1s 14ms/step


array([[0.18726577],
       [0.8258174 ],
       [0.9906883 ],
       [0.1738727 ],
       [0.71170604],
       [0.8171866 ],
       [0.9812652 ],
       [0.97876453],
       [0.95544636],
       [0.09200942]], dtype=float32)

In [ ]:
model_6_preds = tf.squeeze(tf.round(model_6_pred_probs))
model_6_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 1., 1., 1., 1., 1., 0.], dtype=float32)>

In [ ]:
result_6 = cal_results(val_labels , model_6_preds)
result_6

{'accuracy': 82.02099737532808,
 'precision': 0.8210875591779185,
 'recall': 0.8202099737532809,
 'f1': 0.8192635530876636}

## Model 7: TF Hub Pretrained USE with 10% of training data

replicate model 6 and train on 10% of the data

In [ ]:
# NOTE: DATA SPLITS LIKE BELOW LEADS TO DATA LEAKAGE

# Create subsets of 10% of training data
# train_10_percent = train_data_shuffled[["text","target"]].sample(frac=0.1 , random_state = 42)
# train_sentences_10_percent = train_10_percent["text"].to_list()
# train_labels_10_percent = train_10_percent["target"].to_list()

In [ ]:
# CORRECT WAY
train_10_percent_split = int(0.1*len(train_sentences))
train_sentences_10_percent = train_sentences[:train_10_percent_split]
train_labels_10_percent = train_labels[:train_10_percent_split]

In [ ]:

len(train_sentences_10_percent) , len(train_labels_10_percent)

(685, 685)

In [ ]:
train_data_shuffled["target"].value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [ ]:
# Creating a model
model_7 = tf.keras.Sequential([
    encode_layer,
    layers.Dense(64 , activation="relu"),
    layers.Dense(1, activation="sigmoid")
],name ="Model_6_USE")

# Compile the model
model_7.compile(loss ="binary_crossentropy",
                optimizer = tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [ ]:
history_7 = model_7.fit(train_sentences_10_percent,
                        train_labels_10_percent,
                        epochs =5,
                        validation_data=(val_sentences , val_labels),
                        callbacks=[create_tensorboard_callback(SAVE_DIR , "10_percent_expt")])

Saving TensorBoard log files to: model_logs/10_percent_expt/20231210-121752
Epoch 1/5
22/22 [==============================] - 3s 50ms/step - loss: 0.6697 - accuracy: 0.6642 - val_loss: 0.6524 - val_accuracy: 0.7178
Epoch 2/5
22/22 [==============================] - 1s 37ms/step - loss: 0.6045 - accuracy: 0.7927 - val_loss: 0.6001 - val_accuracy: 0.7598
Epoch 3/5
22/22 [==============================] - 1s 52ms/step - loss: 0.5335 - accuracy: 0.8146 - val_loss: 0.5465 - val_accuracy: 0.7690
Epoch 4/5
22/22 [==============================] - 1s 42ms/step - loss: 0.4729 - accuracy: 0.8161 - val_loss: 0.5103 - val_accuracy: 0.7756
Epoch 5/5
22/22 [==============================] - 1s 52ms/step - loss: 0.4282 - accuracy: 0.8263 - val_loss: 0.4957 - val_accuracy: 0.7782


In [ ]:
model_7_pred_probs = model_7.predict(val_sentences)
model_7_pred_probs[:10]

24/24 [==============================] - 1s 11ms/step


array([[0.19222042],
       [0.57312334],
       [0.87963676],
       [0.42336038],
       [0.5233459 ],
       [0.6805737 ],
       [0.8488298 ],
       [0.8036338 ],
       [0.8338327 ],
       [0.1536689 ]], dtype=float32)

In [ ]:
model_7_preds = tf.squeeze(tf.round(model_7_pred_probs))
model_7_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 1., 1., 1., 1., 1., 0.], dtype=float32)>

In [ ]:
result_7 = cal_results(val_labels , model_7_preds)
result_7

{'accuracy': 77.82152230971128,
 'precision': 0.7804499491198106,
 'recall': 0.7782152230971129,
 'f1': 0.7760126933653841}

# Saving and Loading a Trained Model

There are two main formats to save a model to TensorFlow:
1. HDF5 Format
2. The `SavedModel` format

In [ ]:
result_6

{'accuracy': 82.02099737532808,
 'precision': 0.8210875591779185,
 'recall': 0.8202099737532809,
 'f1': 0.8192635530876636}

In [ ]:
# HDF5 Format

model_6.save("model_6.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# Load our model
import tensorflow_hub as hub
loaded_model_6 = tf.keras.models.load_model("model_6.h5",
                                            custom_objects = {"KerasLayer":hub.KerasLayer})

In [ ]:
# Check model
loaded_model_6.evaluate(val_sentences , val_labels)

24/24 [==============================] - 1s 21ms/step - loss: 0.4295 - accuracy: 0.8202


[0.4295121133327484, 0.8202099800109863]

In [ ]:
# Method 2
model_6.save("model_6_TF")

In [ ]:
loaded_6 = tf.keras.models.load_model("model_6_TF")

In [ ]:
loaded_6.evaluate(val_sentences , val_labels)

24/24 [==============================] - 1s 20ms/step - loss: 0.4295 - accuracy: 0.8202


[0.4295121133327484, 0.8202099800109863]

## Finding the most wrong examples

* If our best model still isn't perfect , what examples is it getting wrong?
* And of these wrong examples which ones is it getting *most* wrong

For Example if a sample should have a label of 0 but our model predicts a prediction probability of 0.999 (really close to 1) and vice versa

In [ ]:
# Download a pretrained model to check with our model
! wget https://storage.googleapis.com/ztm_tf_course/08_model_6_USE_feature_extractor.zip
!unzip 08_model_6_USE_feature_extractor.zip

--2023-12-10 14:05:23--  https://storage.googleapis.com/ztm_tf_course/08_model_6_USE_feature_extractor.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.215.207, 173.194.216.207, 173.194.217.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.215.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 960779165 (916M) [application/zip]
Saving to: ‘08_model_6_USE_feature_extractor.zip’

08_model_6_USE_feat 100%[===================>] 916.27M  46.5MB/s    in 27s     

2023-12-10 14:05:50 (34.6 MB/s) - ‘08_model_6_USE_feature_extractor.zip’ saved [960779165/960779165]

Archive:  08_model_6_USE_feature_extractor.zip
   creating: 08_model_6_USE_feature_extractor/
   creating: 08_model_6_USE_feature_extractor/assets/
   creating: 08_model_6_USE_feature_extractor/variables/
  inflating: 08_model_6_USE_feature_extractor/variables/variables.data-00000-of-00001  
  inflating: 08_model_6_USE_feature_extractor/variables/variabl

In [ ]:
# Import previously trained model

model_6_pretrained = tf.keras.models.load_model("08_model_6_USE_feature_extractor")
model_6_pretrained.evaluate(val_sentences , val_labels)

24/24 [==============================] - 1s 11ms/step - loss: 0.4272 - accuracy: 0.8163


[0.42723119258880615, 0.8162729740142822]

In [ ]:
# Make predictions with loaded model from Google Storage
model_6_loaded_pred_probs = model_6_pretrained.predict(val_sentences)
model_6_loaded_preds = tf.squeeze(tf.round(model_6_pred_probs))
model_6_loaded_preds[:10]

24/24 [==============================] - 0s 18ms/step


<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 1., 1., 1., 1., 1., 0.], dtype=float32)>

In [ ]:
# Create DataFrame with validation sentences, validation labels  and best performing model predictions
val_df = pd.DataFrame({"text":val_sentences,
                       "target":val_labels,
                       "pred":model_6_loaded_preds,
                       "pred_probs":tf.squeeze(model_6_loaded_pred_probs)})
val_df.head()

,text,target,pred,pred_probs
0,DFR EP016 Monthly Meltdown - On Dnbheaven 2015...,0,0.0,0.159757
1,FedEx no longer to transport bioterror germs i...,0,1.0,0.747162
2,Gunmen kill four in El Salvador bus attack: Su...,1,1.0,0.988749
3,@camilacabello97 Internally and externally scr...,1,0.0,0.196229
4,Radiation emergency #preparedness starts with ...,1,1.0,0.707808


In [ ]:
# Find the wrong predictions and sort by prediction probabilities

most_wrong = val_df[val_df["target"] != val_df["pred"]].sort_values("pred_probs" , ascending=False)
most_wrong[:10]

,text,target,pred,pred_probs
31,? High Skies - Burning Buildings ? http://t.co...,0,1.0,0.910196
759,FedEx will no longer transport bioterror patho...,0,1.0,0.876982
628,@noah_anyname That's where the concentration c...,0,1.0,0.852300
209,Ashes 2015: AustraliaÛªs collapse at Trent Br...,0,1.0,0.835454
251,@AshGhebranious civil rights continued in the ...,0,1.0,0.827213
393,@SonofLiberty357 all illuminated by the bright...,0,1.0,0.814816
109,[55436] 1950 LIONEL TRAINS SMOKE LOCOMOTIVES W...,0,1.0,0.810840
49,@madonnamking RSPCA site multiple 7 story high...,0,1.0,0.803122
119,@freefromwolves GodsLove &amp; #thankU brother...,0,1.0,0.766901
344,Air Group is here to the rescue! We have 24/7 ...,0,1.0,0.766625


Target Label

* `0` = not disaster
* `1` = disaster

In [ ]:
# Check the false positives(model predicted 1 when 0)

for row in most_wrong[:10].itertuples():
  _ , text , target , pred , pred_prob = row
  print(f"Target: {target} , Pred :{pred} , Prob:{pred_prob}")
  print(f"Text:\n{text}\n")
  print("---\n")

Target: 0 , Pred :1.0 , Prob:0.9101957082748413
Text:
? High Skies - Burning Buildings ? http://t.co/uVq41i3Kx2 #nowplaying

---

Target: 0 , Pred :1.0 , Prob:0.8769820332527161
Text:
FedEx will no longer transport bioterror pathogens in wake of anthrax lab mishaps http://t.co/lHpgxc4b8J

---

Target: 0 , Pred :1.0 , Prob:0.8523001670837402
Text:
@noah_anyname That's where the concentration camps and mass murder come in. 
 
EVERY. FUCKING. TIME.

---

Target: 0 , Pred :1.0 , Prob:0.8354544043540955
Text:
Ashes 2015: AustraliaÛªs collapse at Trent Bridge among worst in history: England bundled out Australia for 60 ... http://t.co/t5TrhjUAU0

---

Target: 0 , Pred :1.0 , Prob:0.8272131681442261
Text:
@AshGhebranious civil rights continued in the 60s. And what about trans-generational trauma? if anything we should listen to the Americans.

---

Target: 0 , Pred :1.0 , Prob:0.814815878868103
Text:
@SonofLiberty357 all illuminated by the brightly burning buildings all around the town!

---

In [ ]:
# Check the false negatives(model predicted 1 when 0)

for row in most_wrong[-10:].itertuples():
  _ , text , target , pred , pred_prob = row
  print(f"Target: {target} , Pred :{pred} , Prob:{pred_prob}")
  print(f"Text:\n{text}\n")
  print("---\n")

Target: 1 , Pred :0.0 , Prob:0.06730345636606216
Text:
@DavidVonderhaar At least you were sincere ??

---

Target: 1 , Pred :0.0 , Prob:0.055075839161872864
Text:
@willienelson We need help! Horses will die!Please RT &amp; sign petition!Take a stand &amp; be a voice for them! #gilbert23 https://t.co/e8dl1lNCVu

---

Target: 1 , Pred :0.0 , Prob:0.054603397846221924
Text:
Lucas Duda is Ghost Rider. Not the Nic Cage version but an actual 'engulfed in flames' badass. #Mets

---

Target: 1 , Pred :0.0 , Prob:0.05459701269865036
Text:
going to redo my nails and watch behind the scenes of desolation of smaug ayyy

---

Target: 1 , Pred :0.0 , Prob:0.04963730648159981
Text:
You can never escape me. Bullets don't harm me. Nothing harms me. But I know pain. I know pain. Sometimes I share it. With someone like you.

---

Target: 1 , Pred :0.0 , Prob:0.04391849413514137
Text:
@SoonerMagic_ I mean I'm a fan but I don't need a girl sounding off like a damn siren

---

Target: 1 , Pred :0.0 , Prob:0

## Making Predictions on the test dataset

In [ ]:
# Making predictions on text dataset and visualizing them

test_sentences = val_df["text"].to_list()
test_samples = random.sample(test_sentences ,10)

for test_sample in test_samples:
  pred_prob = tf.squeeze(model_6_pretrained.predict([test_sample]))
  pred = tf.round(pred_prob)

  print(f"Pred:{int(pred)} , Prob:{pred_prob}")
  print(f"Text:\n{test_sample}\n")
  print("---\n")

1/1 [==============================] - 0s 80ms/step
Pred:1 , Prob:0.9855031371116638
Text:
Wreckage 'Conclusively Confirmed' as From MH370: Malaysia PM: Investigators and the families of those who were... http://t.co/gRh7zLK979

---

1/1 [==============================] - 0s 67ms/step
Pred:0 , Prob:0.09586482495069504
Text:
@SetZorah dad why dont you claim me that mean that not right we look the same same eyes same blood same xbox 360 SMH -.-

---

1/1 [==============================] - 0s 68ms/step
Pred:0 , Prob:0.22207221388816833
Text:
@chromsucks don't drown

---

1/1 [==============================] - 0s 66ms/step
Pred:0 , Prob:0.05870313197374344
Text:
The one thing I like about here is thunder .. Heheh

---

1/1 [==============================] - 0s 66ms/step
Pred:0 , Prob:0.36174020171165466
Text:
@ColdMpress You up to commiting mass murder tonight?

---

1/1 [==============================] - 0s 63ms/step
Pred:0 , Prob:0.3821880519390106
Text:
@BookTubeAThon A world in which p

# Speed/Score TradeOff

In [ ]:
result_6

{'accuracy': 82.02099737532808,
 'precision': 0.8210875591779185,
 'recall': 0.8202099737532809,
 'f1': 0.8192635530876636}

In [ ]:
# Lets make a function to measure time of prediction

import time

def pred_timer(model , samples):
  """
  Times how long a model takes to make predictions on samples
  """

  start_time = time.perf_counter() # Get start Time
  model.predict(samples)
  end_time = time.perf_counter()
  total_time = end_time - start_time

  time_per_pred = total_time /len(samples)

  return total_time , time_per_pred



In [ ]:
# Calculate time taken to predict
model_6_total_pred_time , model_6_time_per_pred = pred_timer(model_6_pretrained , val_sentences)
model_6_total_pred_time , model_6_time_per_pred

24/24 [==============================] - 0s 13ms/step


(0.7343909270002769, 0.0009637676207352715)

In [ ]:
model_6_pretrained_results = cal_results(val_labels , model_6_loaded_preds)
model_6_pretrained_results


{'accuracy': 82.02099737532808,
 'precision': 0.8210875591779185,
 'recall': 0.8202099737532809,
 'f1': 0.8192635530876636}